### 分段函数（Piecewise Functions）

在前一章[《[数字图像学笔记] 4.直方图变换1》](https://seagochen.blog.csdn.net/article/details/110204387)中所提到几种针对直方图的变化方法，基本上都是针对 $[0, 255]$ 全部灰度值的变化，然后我们在实际的使用中也许并不需要对全部的灰度值进行变化，比如我们只想调整其中一部分的强度，或者只显示其中的一部分。

那么这种方法就是分段线性变化，有的地方也称为分段滤波函数，或者直接称呼它的数学定义**分段函数**。依据我们小学知道的数学定义，分段函数可以有多个不同的函数组成。

所以写成公式的话，就表示成如下的样子：

$$
f(X) = \left\{\begin{matrix}
F_1 \left ( x_0 \right ) & 0 \leq  x_0 < I_0 \\ 
F_2 \left ( x_1 \right ) & I_0 \leq x_1 < I_1 \\
\cdots & \cdots \\
F_n \left ( x_n \right ) & I_n \leq x_n \leq 255
\end{matrix}\right.
$$

### 分段线性变化函数（Piecewise Transformation Function）

#### 局部对比度拉伸（Contrast Stretching）

在冈萨雷斯的教材里，提到了第一个简单的Sample，好像是把花粉还是种子的图片进行对比度拉伸。原图是一张对比度非常低的图片，很多图片中的细节看不清楚，因此我们使用一个简单的分段函数，对原图进行对比度拉伸。

$$
f(X) = \left\{\begin{matrix}
0.25 * x & 0 \leq  x < 90 \\ 
1.25 * x & 90 \leq x < 160 \\
0.25 * x & 160 \leq x_n \leq 255
\end{matrix}\right.
$$

#### 灰度级分层（Intensity-level slicing）

说到底这个方法也是分段线性函数的一种变形形式，也就是把我们感兴趣的灰度范围高亮、增强，把不喜欢的隐藏掉或者降低它的强度值。

$$
f(X) = \left\{\begin{matrix}
F_1 \left ( x_0 \right ) & 0 \leq  x_0 < I_0 \\ 
F_2 \left ( x_1 \right ) & I_0 \leq x_1 < I_1 \\
\cdots & \cdots \\
F_n \left ( x_n \right ) & I_n \leq x_n \leq 255
\end{matrix}\right.
$$

假设，比如对于分段函数来说，我们令范围 $[0, I_0]$ 和 $F_m  \in [I_m, I_n]$ 及 $[I_n, 255]$ 为0，也就是强调其中的$[I_{m-1}, I_m]$这一段，以上的分段函数就可以写作：

$$
f(X) = \left\{\begin{matrix}
F_1 \left ( x_0 \right ) = 0 & 0 \leq  x_0 < I_0 \\ 
\cdots \\
F_{m-1} \left ( x_m \right ) = f_m & I_{m-1} \leq x_{m-1} < I_m \\
\cdots \\
F_n \left ( x_n \right ) = 0 & I_n \leq x_n \leq 255
\end{matrix}\right.
$$

当然也可以只增强兴趣区域的值，所以你能看到两个不同的分段增强函数的映射曲线，这里我就不写代码示例了，其实实现方法和上面的那个拉伸方法是相似的。

#### 比特面分层（Bit-plane slicing）

这个简单的说一下，主要是用于图像压缩的一种方法。对于一个灰度像素，它的数字通常是由8个比特组成的，你可以想象是由包含8个比特的数字组成的：

$$[(2^7)n_7, (2^6)n_6, (2^5)n_5, (2^4)n_4, (2^3)n_3, (2^2)n_2, (2^1)n_1, (2^0)n_0 ]$$

其中的 $n$ 表示的是一个数位开关，它只有开合两种状态，因此对于一个在$[0, 255]$范围的数来说，就有8个比特位，如果我们完整的记录一个比特数，从存储空间上来说，就需要记录8个开关的状态，比如[1, 0, 0, 1, 1, 0, 0, 1]。

对一个灰度的还原方法因此就成了这样

$$P = (2^7)n_7 + (2^6)n_6 + (2^5)n_5+ (2^4)n_4 + (2^3)n_3 + (2^2)n_2 + (2^1)n_1 + (2^0)n_0 $$

实际上对于一张图片来说，它有时候保留的有效开关信息其实很少，比如说一张天空的照片，它的有效像素信息可能几种在 $(n_0, n_1, n_4)$，而在$n_7$有少量信息，而其他位上则基本没有信息。如果为了获得最大压缩率，那么其实我们就可以只保留$(n_0, n_1, n_4)$这三个开关的信息，为了更好的说明，我这里用矩阵表示一下这个过程：

$$
I = \begin{bmatrix}
n_3 & n_2 & n_1 & n_0 \\
0 & 0 & 1 & 1 \\ 
0 & 1 & 0 & 1 \\ 
0 & 1 & 1 & 1 \\ 
0 & 0 & 1 & 1
\end{bmatrix}
$$

例如以上的矩阵，一共有4个点，$n_3$ 由于没有数据，所以在对数据进行分层的时候，我们就可以把这一层舍弃，而不损失任何精度和细节。如果我们只保留$n_1$和$n_0$两层比特层，如果在保留一定的细节情况下，那么就可以在理论上把数据压缩一半：

$$
I = \begin{bmatrix}
n_1 & n_0 \\
1 & 1 \\ 
0 & 1 \\ 
1 & 1 \\ 
1 & 1
\end{bmatrix}
$$

对于高级语言来说，要把某个数转化位比特，通常需要**与运算(and operation)**，具体这里就不实现了。

### 采样函数（Sampling Functions）

从上面的这些例子中，我们发现，能够对于原图像的灰度图的修改，很大程度上用到了某种函数，无论是拉伸，还是其他，我们都是通过 $I_o = I_i \bigodot f_s$ 这样一种形式进行的扩展。这个式子中的$f_s$ 就是采样函数。

$\bigodot$ 在这里，我们不明确指明原始数据与采样函数的计算方式，它可以是点乘，可以是矩阵乘，也可以是卷积，甚至就像上面介绍的这些分段函数一样，或者单纯的加一些常数。

![在这里插入图片描述](https://img-blog.csdnimg.cn/20201207222337554.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3BvaXNvbmNocnk=,size_16,color_FFFFFF,t_70#pic_center)

例如对于原始数据，在图中表示红色曲线的部分，其采样函数$f_s$为正态分布曲线，数学上写作：

$$
f_s = \frac{1}{\sqrt{2 \pi \delta^2}}e(-\frac{(x-\mu)^2}{2\delta^2})
$$

那么对于这样一个数据，当$\bigodot$表示为元素乘的时候，采样函数为正态分布，每一个像素点的新输出的计算方式即： 
$$
{I_1}'= I_1 \cdot f_s(P_1) \\
{I_2}'= I_2 \cdot f_s(P_2) \\
\cdots \\
{I_n}'= I_n \cdot f_s(P_n) \\
$$